In [1]:
import torch
import torch.nn as nn
from torchsummaryX import summary
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt

In [2]:
# img_path = '/home/xieliang/Data/NTS-Net/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg'
# img = plt.imread(img_path)
# plt.imshow(img)
# plt.show()

In [3]:
def init_params(net):
    for name, params in net.named_parameters():
        if 'weight' in name:
            nn.init.uniform_(params.data)
        if 'bias' in name:
            nn.init.constant_(params.data, 0)
            

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3,1,0)
        self.pool = nn.AdaptiveAvgPool2d((7,7))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32*7*7,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [4]:
net = Net()
net.apply(init_params)
optimizer = optim.Adam(net.fc.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [20, 40])
writer = SummaryWriter('/home/xieliang/Data/sign-language-recognition/log/test_pytorch')

# for name, params in net.named_parameters():
#     if 'fc' in name:
#         params.requires_grad=False
#     print(name)
#     print(params)
    
# for name, param in net.named_parameters():
#     if 'conv1' in name:
#         param.requires_grad = False
#     print(name)
#     print(param)
# print(net)
# summary(net, torch.zeros(1,3,28,28))

In [5]:
def train(net, optimizer, criterion):
        
    for i in range(900,1000):
        net.train()

        optimizer.zero_grad()
        inputs = torch.zeros(2,3,28,28)
        outs = net(inputs)
        loss = criterion(outs, torch.tensor([0,1]))
        
        for name, params in net.named_parameters():
            if 'conv1.weight' in name:
                writer.add_histogram('conv1_weights', params, i)
            if 'conv1.bias' in name:
                writer.add_histogram('conv1_bias', params, i)
            if 'fc.weight' in name:
                writer.add_histogram('fc_weitghs', params, i)
            if 'fc.bias' in name:
                writer.add_histogram('fc_bias', params, i)

        loss.backward()
        optimizer.step()

In [9]:
print(lr_scheduler.get_lr())

[0.0001]


In [4]:
import numpy as np
a = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
b = [np.random.randint(i,i+4) for i in range(0,20,4)]
print(b)

[3, 5, 10, 12, 17]
